## Code pour le Baseline Neural Net

In [4]:

from scipy.integrate import solve_ivp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout

import jax
import jax.numpy as jnp
import numpy as np
from jax.experimental.ode import odeint
import matplotlib.pyplot as plt
from functools import partial # reduces arguments to function by making some subset implicit

from jax.example_libraries import stax
from jax.example_libraries import optimizers

# visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from functools import partial
import proglog

import optax

ModuleNotFoundError: No module named 'tensorflow'

### Importation des fonctions pour générer les trajectoires

In [ ]:
# Définir la durée et la fonction de génération de trajectoire
duree = np.linspace(0, 20, num=401)

def generate_trajectory(initial):
    trajectory = jax.device_get(solve_analytical(initial, duree))
    return trajectory

# Fonction pour générer le dataset
def generate_dataset(num_samples, duree):
    dataset = []
    for i in range(num_samples):
        # Générer des conditions initiales aléatoires
        theta1 = np.random.uniform(-np.pi/2, np.pi/2)
        omega1 = 0
        theta2 = np.random.uniform(-np.pi/2, np.pi/2)
        omega2 = 0
        initial = np.array([theta1, theta2, omega1, omega2], dtype=np.float32)

        try:
            # Générer la trajectoire
            trajectory = generate_trajectory(initial)  # Forme: (301, 4)

            # Ajouter au dataset
            dataset.append({
                'initial_conditions': initial,
                'trajectory': trajectory
            })
        except RuntimeError:
            # Ignorer les échecs d'intégration
            continue

        if (i+1) % 1000 == 0:
            print(f"{i+1} échantillons générés...")
    return dataset

# Générer le dataset (peut prendre du temps en fonction de num_samples)
num_samples = 5000
dataset = generate_dataset(num_samples, duree)
print(f'Dataset généré avec {len(dataset)} échantillons.')

Si on veut sauvegarder le dataset

In [ ]:
#Sauvegarder le dataset
with open('double_pendulum_dataset.pkl', 'wb') as f:
    pickle.dump(dataset, f)

#Charger le dataset (si nécessaire)
with open('double_pendulum_dataset.pkl', 'rb') as f:
    dataset = pickle.load(f)

Visualisons une donné du dataset pour voir si c'est ok.

In [ ]:
initial = np.array([np.pi/4, np.pi/4, 0, 0], dtype=np.float32)
trajectory = jax.device_get(solve_analytical(initial, duree))

plt.plot(duree, trajectory[:,1])

traj_bis = generate_trajectory(initial)
plt.plot(duree, traj_bis[:,1], '+')

In [ ]:
import matplotlib.pyplot as plt

# Supposons que votre dataset est déjà généré et stocké dans la variable `dataset`
# Si vous avez sauvegardé le dataset, décommentez les lignes suivantes pour le charger
# with open('double_pendulum_dataset.pkl', 'rb') as f:
#     dataset = pickle.load(f)

# Vérifiez si le dataset contient des échantillons
if len(dataset) == 0:
    print("Le dataset est vide. Veuillez vérifier la génération des données.")
else:
    # Choisir un échantillon à visualiser
    sample_idx = 100  # Vous pouvez changer cet index pour visualiser d'autres échantillons
    sample = dataset[sample_idx]
    initial_conditions = sample['initial_conditions']
    trajectory = sample['trajectory']  # Forme: (301, 4)

    print(f"Conditions initiales : {initial_conditions}")
    #print(f"Trajectoire : {trajectory}")

    # Extraire les valeurs de theta1, omega1, theta2, omega2
    theta1 = trajectory[:, 0]
    omega1 = trajectory[:, 2]
    theta2 = trajectory[:, 1]
    omega2 = trajectory[:, 3]

    # Tracer les angles et les vitesses angulaires au fil du temps
    plt.figure(figsize=(14, 10))

    plt.subplot(2, 1, 1)
    plt.plot(duree, theta1, label=r'$\theta_1$ (rad)')
    plt.plot(duree, theta2, label=r'$\theta_2$ (rad)')
    plt.title('Angles du Double Pendule en Fonction du Temps')
    plt.xlabel('Temps (s)')
    plt.ylabel('Angle (rad)')
    plt.legend()
    plt.grid(True)

    plt.subplot(2, 1, 2)
    plt.plot(duree, omega1, label=r'$\omega_1$ (rad/s)')
    plt.plot(duree, omega2, label=r'$\omega_2$ (rad/s)')
    plt.title('Vélocités Angulaires du Double Pendule en Fonction du Temps')
    plt.xlabel('Temps (s)')
    plt.ylabel('Vitesse Angulaire (rad/s)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Optionnel : Tracer les trajectoires des angles entre eux
    plt.figure(figsize=(8, 6))
    plt.plot(theta1, theta2, label='Trajectoire Angulaire')
    plt.title('Trajectoire Angulaire du Double Pendule')
    plt.xlabel(r'$\theta_1$ (rad)')
    plt.ylabel(r'$\theta_2$ (rad)')
    plt.legend()
    plt.grid(True)
    plt.show()

On prépare les données pour l'apprentissage

In [ ]:
# Préparer les données
X = np.array([sample['initial_conditions'] for sample in dataset])  # Forme: (num_samples, 4)
y = np.array([sample['trajectory'] for sample in dataset])         # Forme: (num_samples, 301, 4)
print(f'X shape: {X.shape}, y shape: {y.shape}')

# Normalisation
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

num_samples, timesteps, features = y.shape
y_reshaped = y.reshape(-1, features)  # Forme: (num_samples * timesteps, 4)

scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y_reshaped).reshape(num_samples, timesteps, features)

# Séparation train/test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
print(f'Entraînement: {X_train.shape}, Test: {X_test.shape}')

# Définir les dimensions
input_dim = X_train.shape[1]          # 4
output_timesteps = y_train.shape[1]   # 301
output_features = y_train.shape[2]    # 4

On construit l'architecture du NN

In [ ]:
# Construire le modèle
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(input_dim,)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(RepeatVector(output_timesteps))
model.add(LSTM(256, return_sequences=True, activation='tanh'))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(TimeDistributed(Dense(output_features)))

# Compiler le modèle
model.compile(optimizer='adam', loss='mse')
model.summary()

# Entraîner le modèle
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
)

# Visualiser l'entraînement
plt.plot(history.history['loss'], label='Entraînement')
plt.plot(history.history['val_loss'], label='Validation')
plt.legend()
plt.xlabel('Époques')
plt.ylabel('MSE Loss')
plt.title('Entraînement du Modèle')
plt.show()

# Évaluer le modèle
test_loss = model.evaluate(X_test, y_test)
print(f'Loss sur le jeu de test : {test_loss}')

# Faire des prédictions
predictions = model.predict(X_test)

# Inverser la normalisation
predictions_inv = scaler_y.inverse_transform(predictions.reshape(-1, output_features)).reshape(predictions.shape)
y_test_inv = scaler_y.inverse_transform(y_test.reshape(-1, output_features)).reshape(y_test.shape)

# Visualiser une séquence
sequence_idx = 0

plt.figure(figsize=(14, 8))
labels = ['θ1', 'θ2', 'ω1', 'ω2']

for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.plot(y_test_inv[sequence_idx, :, i], label=f'{labels[i]} réel')
    plt.plot(predictions_inv[sequence_idx, :, i], label=f'{labels[i]} prédit', alpha=0.7)
    plt.legend()
    plt.xlabel('Pas de temps')
    plt.ylabel(labels[i])
    plt.title(f'Comparaison de {labels[i]} réel et prédit')

plt.tight_layout()
plt.show()

On fait les prédictions avec des nouvelles data

In [ ]:
# 1. Définir vos propres conditions initiales
custom_initial = np.array([np.pi/4, 0, 0, 0], dtype=np.float32)
print("Conditions Initiales Personnalisées:", custom_initial)

# 2. Normaliser les conditions initiales
custom_initial_scaled = scaler_X.transform(custom_initial.reshape(1, -1))  # Forme: (1, 4)
print("Conditions Initiales Normalisées:", custom_initial_scaled)

# 3. Faire la prédiction
predicted_scaled = model.predict(custom_initial_scaled)  # Forme: (1, 301, 4)
print("Prédiction Normalisée Shape:", predicted_scaled.shape)

# 4. Inverser la normalisation
predicted_inv = scaler_y.inverse_transform(predicted_scaled.reshape(-1, 4)).reshape(predicted_scaled.shape)  # Forme: (1, 301, 4)
print("Prédiction Inversée Shape:", predicted_inv.shape)

# 5. Visualiser la trajectoire prédite
duree = np.linspace(0, 20, num=301)  # Assurez-vous que 'duree' est défini

trajectory_pred = predicted_inv[0]  # Forme: (301, 4)

# 6. (Optionnel) Comparer avec une trajectoire réelle
# Générer la trajectoire réelle
actual_trajectory = generate_trajectory(custom_initial)  # Forme: (301, 4)
print("Trajectoire Réelle Shape:", actual_trajectory.shape)

plt.figure(figsize=(14, 10))

for i in range(4):
    plt.subplot(2, 2, i+1)
    plt.plot(duree, actual_trajectory[:, i], label='Réel', color='blue', alpha=0.4)
    plt.plot(duree, trajectory_pred[:, i], label='Prédit', color='orange', alpha=0.7)
    plt.legend()
    plt.xlabel('Temps (s)')
    plt.ylabel(f'{labels[i]} ({units[i]})')
    plt.title(f'Comparaison de {labels[i]} Réel et Prédit')
    plt.grid(True)

# Ajouter les conditions initiales comme une annotation globale
plt.suptitle(
    f'Conditions Initiales:\n'
    f'θ1 = {custom_initial[0]:.2f} rad, θ2 = {custom_initial[1]:.2f} rad\n'
    f'ω1 = {custom_initial[2]:.2f} rad/s, ω2 = {custom_initial[3]:.2f} rad/s',
    fontsize=16,
    y=0.95
)

plt.tight_layout(rect=[0, 0, 1, 0.93])  # Ajuster l'espace pour la super-titre
plt.show()

Vérifions l'énergie

In [ ]:
def check_energy(th1, th2, w1, w2, m1=1, m2=1, l1=1, l2=1, g=9.8):

  T = 0.5*(m1 + m2)*l1**2*w1**2 + 0.5*m2*l2**2*w2**2 + m2*l1*l2*w1*w2*jnp.cos(th1 - th2)
  V = (m1+m2)*g*l1*jnp.cos(th1) + m2*g*l2*jnp.cos(th2)

  return T - V

In [ ]:
energy_analytique = check_energy(actual_trajectory[:,0], actual_trajectory[:,1], actual_trajectory[:,2], actual_trajectory[:,3])
energy_pred = check_energy(trajectory_pred[:,0], trajectory_pred[:,1], trajectory_pred[:,2], trajectory_pred[:,3])
plt.plot(duree, energy_analytique, label='Energie Analytique')
plt.plot(duree, energy_pred, label='Energie Baseline NN')

plt.xlabel(r'Temps (s)')
plt.ylabel(r'Energie (J)')
plt.legend()